# Find Abbreviations
This notebook scans the text collection for Abbreviations (tokens that contain a period) and outputs all of them in a dictionary/text file which can later be reviewed and post processed using, for example, string functions in Notepad++:

- Edit->Convert Case to->lowercase
- Edit->Line Operations->Sort Lines Lexicographically Ascending
- Edit->Line Operations->Remove Duplicate Lines

## Get Sample Text Data fom Local Storage

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
sourcePath = "/gdrive/MyDrive/GermanDataSets/CourtRulings/100_SentenceSplits/"
!ls {sourcePath.replace(' ', '\ ')} -lha

total 31M
-rw------- 1 root root 187K Aug 18 12:48 '00 Test Standard Sentence Detection.ipynb'
-rw------- 1 root root 2.8M Aug 18 12:53  100_2019-02-19_oldp_cases_textout.csv
-rw------- 1 root root 2.9M Aug 15 11:48  100_2019-02-19_oldp_cases_textout.json
-rw------- 1 root root  14K Aug 18 13:19 '10 Train Custom Sentence Detection.ipynb'
-rw------- 1 root root  25M Aug 15 09:56  1k_2019-02-19_oldp_cases_textout.json
-rw------- 1 root root  14K Aug 18 13:40 '20 Find Abbreviations.ipynb'


# Setup Spark

In [ ]:
# This is only to setup PySpark and Spark NLP on Colab
!wget http://setup.johnsnowlabs.com/colab.sh -O - | bash

--2021-08-18 13:43:34--  http://setup.johnsnowlabs.com/colab.sh
Resolving setup.johnsnowlabs.com (setup.johnsnowlabs.com)... 51.158.130.125
Connecting to setup.johnsnowlabs.com (setup.johnsnowlabs.com)|51.158.130.125|:80... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh [following]
--2021-08-18 13:43:34--  https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1608 (1.6K) [text/plain]
Saving to: ‘STDOUT’

-                     0%[                    ]       0  --.-KB/s               setup Colab for PySpark 3.1.2 and Spark NLP 3.2.1
-                   100%[==

In [ ]:
from pyspark.ml import PipelineModel
from sparknlp.annotator import *
from sparknlp.base import *
import sparknlp

spark = sparknlp.start()

print("Spark version:", spark.version)
print("Spark NLP version:", sparknlp.version())

Spark version: 3.1.2
Spark NLP version: 3.2.1


# Read Data & Test Split Sentences with Multilingual Spark NLP Model

In [ ]:
df = spark.read.json(sourcePath + '1k_2019-02-19_oldp_cases_textout.json')

df.printSchema()
df.count()

root
 |-- id: long (nullable = true)
 |-- text_Content: string (nullable = true)



1000

# Convert to Text File

In [ ]:
def ConvertText(str):
    resStr=""
    resStr = str.replace('\r', ' ').replace('\n', ' ').replace('\t', ' ')

    return resStr


In [ ]:
testText = 'Dies ist eine Zeile mit einem Enter\nalso eigentlich 2 Zeilen.'
convertedText = ConvertText(testText)

print (f"testText: '{testText}'\n")
print (f"convertedText: '{convertedText}'")

testText: 'Dies ist eine Zeile mit einem Enter
also eigentlich 2 Zeilen.'

convertedText: 'Dies ist eine Zeile mit einem Enter also eigentlich 2 Zeilen.'


In [ ]:
import pyspark.sql.functions as F

""" Converting function to UDF """
ConvertTextUDF = F.udf(lambda z: ConvertText(z),StringType())

In [ ]:
convertedTexDF = df.select(F.col("id"), ConvertTextUDF(F.col("text_Content")).alias("text_conv") )
convertedTexDF.show(truncate=False)

+------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
# Save result as text - one line per document
convertedTexDF.select('text_conv').coalesce(1).write.format("text").option("header", "false").mode("append").save("output.txt")

In [ ]:
ls output.txt/ -lha

total 25M
drwxr-xr-x 2 root root 4.0K Aug 18 15:10 ./
drwxr-xr-x 1 root root 4.0K Aug 18 15:10 ../
-rw-r--r-- 1 root root  25M Aug 18 15:10 part-00000-a72621e0-923a-4724-ac60-1d6f1290bf03-c000.txt
-rw-r--r-- 1 root root 198K Aug 18 15:10 .part-00000-a72621e0-923a-4724-ac60-1d6f1290bf03-c000.txt.crc
-rw-r--r-- 1 root root    0 Aug 18 15:10 _SUCCESS
-rw-r--r-- 1 root root    8 Aug 18 15:10 ._SUCCESS.crc


In [ ]:
!mv output.txt/part-00000-a72621e0-923a-4724-ac60-1d6f1290bf03-c000.txt ./ConvertedText.txt

In [ ]:
# Sanity check
!head -n 50 ./ConvertedText.txt

 Tenor Als funktional zuständig wird die allgemeine Zivilkammer bestimmt. Gründe I. Die in München ansässige Klägerin, ein Versicherungsunternehmen, begehrt nach Abgabe an das im Mahnbescheid als Streitgericht bezeichnete Landgericht Augsburg mit Anspruchsbegründung vom 12.2.2018 von dem im Bezirk des Landgerichts Augsburg wohnhaften Beklagten, einem Versicherungsvermittler, Rückzahlung von Provisionsvorschüssen i.H.v. 6.150,23 € wegen vom Beklagten vermittelter, aber stornierter bzw. reduzierter Versicherungen. Die Parteien schlossen am 1.4.2009 einen Versicherungsagenturvertrag, wonach der Beklagte ausschließlich für die Klägerin im Wesentlichen die Erschließung neuer Kundenkreise und die Pflege der Bestandskunden übernahm. Die Klägerin stützt ihren Anspruch auf Ziff. 4 der nach dem Agenturvertrag geltenden Allgemeinen Provisionsbedingungen, wonach eine Abschlussprovision zurückbelastet wird, soweit ein Versicherungsvertrag storniert oder reduziert wird. Das Verfahren wurde zunächst 

In [ ]:
def CollectAbbr(abbreviations, text, _digits):

  #split string by single space
  chunks = text.split(' ')

  #go through tokens and keep note of thos that contain a period '.'
  for chunk in chunks:
    # last character should be period and no brackets or numbers please
    if chunk.find('.') == len(chunk) - 1 and chunk.find('(') == -1 and chunk.find(')') == -1 and bool(_digits.search(chunk)) == False:
      if chunk not in abbreviations:
        abbreviations.add(chunk)

  return abbreviations

In [ ]:
import re

abbrSet = set()
_digits = re.compile('\d')

# Using readlines()
file1 = open('./ConvertedText.txt', 'r')
Lines = file1.readlines()
 
count = 0
# Strips the newline character
for line in Lines:
    count += 1
    abbrSet = CollectAbbr(abbrSet, line, _digits)

    if count < 3:
      print("Line{}: {}".format(count, line.strip()))

print(f"count: {count} '{abbrSet}''")


Line1: Tenor Als funktional zuständig wird die allgemeine Zivilkammer bestimmt. Gründe I. Die in München ansässige Klägerin, ein Versicherungsunternehmen, begehrt nach Abgabe an das im Mahnbescheid als Streitgericht bezeichnete Landgericht Augsburg mit Anspruchsbegründung vom 12.2.2018 von dem im Bezirk des Landgerichts Augsburg wohnhaften Beklagten, einem Versicherungsvermittler, Rückzahlung von Provisionsvorschüssen i.H.v. 6.150,23 € wegen vom Beklagten vermittelter, aber stornierter bzw. reduzierter Versicherungen. Die Parteien schlossen am 1.4.2009 einen Versicherungsagenturvertrag, wonach der Beklagte ausschließlich für die Klägerin im Wesentlichen die Erschließung neuer Kundenkreise und die Pflege der Bestandskunden übernahm. Die Klägerin stützt ihren Anspruch auf Ziff. 4 der nach dem Agenturvertrag geltenden Allgemeinen Provisionsbedingungen, wonach eine Abschlussprovision zurückbelastet wird, soweit ein Versicherungsvertrag storniert oder reduziert wird. Das Verfahren wurde zun

In [ ]:
abbr_sorted = sorted(abbrSet)
abbr_sorted

['',
 '"Befähigung".',
 '"E".',
 '"Eingriffsrecht".',
 '"F.',
 '"Gesetz".',
 '"I.',
 '"Riegelwirkung".',
 '"Schwarzbau".',
 '"Storchengedicht".',
 '"Strukturbezogenheit".',
 '"Verhandlungsmasse".',
 '"Widerspruch/Beschwerde".',
 '"a.',
 '"alternativlos".',
 '"anfallen".',
 '"verstaatlichen".',
 '"vorliege".',
 '"zurückgestellt".',
 '#####.',
 '#####/####.',
 '%.',
 "'PEBB§Y'.",
 '-,jew.',
 '-.',
 '-AE.',
 '-Gymnasiums.',
 '-Rn.',
 '-Straße.',
 '-Universität“.',
 '-\xa0d.',
 '-\xa0vgl.',
 '.',
 '/Aserbaidschan.',
 '/K.',
 '<ABl.',
 '<GVBl.',
 '<SächsGVBl.',
 'A-Stadt.',
 'A-Tumoren.',
 'A.',
 'ABl.',
 'AC-Gelenkes.',
 'ADHS.',
 'AEUV.',
 'AG.',
 'AGG.',
 'AMG.',
 'AMS.',
 'AO.',
 'APSO-INGI.',
 'ARegV.',
 'AU-Feststellungen.',
 'AVB.',
 'AVL.',
 'Aarhus.',
 'Ab.',
 'Abartigkeit.',
 'Abbldg.',
 'Abbruchs.',
 'Abdr.',
 'Abfallbeseitigung.',
 'Abfallbewirtschaftungsgebühr.',
 'Abfalleigenschaft.',
 'Abfalls.',
 'Abfallverarbeitungsdienstleistungen.',
 'Abflugtermin.',
 'Abfälle.',
 'Abfäll

In [ ]:
# Save set of candidate abbreviations to file storage for review
with open(sourcePath + 'abbreviations.txt','w') as f:
  for abbr in abbr_sorted:
    f.write(abbr + '\n')  # set of numbers & a tuple